# To do
- optElem[21:]
- set indexes
- figure out how to make it useful

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [243]:
infos = []
collated_timetable = pd.DataFrame()

driver = webdriver.Chrome()
driver.get('https://wish.wis.ntu.edu.sg/webexe/owa/aus_schedule.main')

optElem = driver.find_elements_by_tag_name('option')
courses = optElem[21:] # change this, involves sem form 1st option

for i, course in enumerate(courses):
    # remove headers that leads to nothing
    if course.text == ' ' or '---' in course.text:
        pass
    else:
        # select module and click load
        course_name = course.text
        
        # click on one of the course
        driver.find_element_by_xpath(f'//*[@id="ui_body_container"]/table/tbody/tr/td[2]/select[2]/option[{i+1}]').click() 
        
        # click load class schedule
        driver.find_element_by_xpath('//*[@id="ui_body_container"]/table/tbody/tr/td[2]/input').click()

        # switch to 2nd window
        driver.switch_to.window(driver.window_handles[1])
        
        # get course name
        course_name = driver.find_element_by_xpath('/html/body/center/center/font[1]/b/b/font').text.split('\n')[1]
        print(f'\nScanning couse: {course_name}')
        
        # scrape module details
        for no, module in enumerate(driver.find_elements_by_tag_name('table')):
            
            # retrieve info column
            if no%2 == 0:
                info = []
                df = pd.read_html(driver.find_elements_by_tag_name('table')[no].get_attribute('outerHTML'))[0]
                                                
                # get column details
                info = list(df.iloc[0])
                
                # add flag for PT
                mod_code = f'{info[0]} {"PT" if "PartTime" in course_name else ""}'
                mod_name = f'{info[1]} {"PT" if "PartTime" in course_name else ""}'
                
                info[0] = mod_code
                info[1] = mod_name
                print(f'\tScrapping module for {mod_code}: {mod_name}')

                # check for prereq (row)
                try:
                    info.append(' '.join(df[1].iloc[1:]))
                except TypeError:
                    pass

                infos.append(info)
            
            # retrieve timetable
            elif no%2 == 1:
                timetable = pd.read_html(driver.find_elements_by_tag_name('table')[no].get_attribute('outerHTML'))[0]
                
                # replace 1st column as headers, NOTE: throws FutureWarning
                timetable.columns = timetable.iloc[0]
                timetable.drop([0], axis = 0, inplace = True)

                # fill all na in INDEX and REMARK
                timetable.fillna(method='ffill', inplace = True)

                # drop all lectures since duplicates
                timetable.drop_duplicates(subset = ['TYPE','GROUP','DAY','TIME','VENUE'], inplace = True)
                
                # set new column as module code
                timetable['MODULE'] = mod_code
                timetable['NAME'] = mod_name
                
                # change index number of LEC, ignore if empty e.g. online courses
                try:
                    timetable.loc[timetable.TYPE == 'LEC/STUDIO', 'INDEX'] = 'LEC'
                except TypeError:
                    pass
                
                collated_timetable = collated_timetable.append(timetable)
                
        # close and switch back to first window
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        


Scanning couse: Accountancy Year 1 (GA)
	Scrapping module for AB0601 : COMMUNICATION MANAGEMENT FUNDAMENTALS 
	Scrapping module for AB0901 : PRINCIPLES OF ECONOMICS: A SINGAPORE PERSPECTIVE* 
	Scrapping module for AB1000 : CAREER FOUNDATIONS I 
	Scrapping module for AB1201 : FINANCIAL MANAGEMENT* 
	Scrapping module for AB1202 : STATISTICS & ANALYSIS* 
	Scrapping module for AB1301 : BUSINESS LAW* 
	Scrapping module for AB1401 : TECHNOLOGICAL INNOVATIONS & DEVELOPMENTS* 
	Scrapping module for AB1402 : FOUNDATIONAL EXCEL* 
	Scrapping module for AB1501 : MARKETING* 


C:\Users\JIanHui\Anaconda3\lib\site-packages\pandas\core\ops.py:1167: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


	Scrapping module for AB1601 : ORGANISATIONAL BEHAVIOUR & DESIGN* 
	Scrapping module for AC1101 : ACCOUNTING I 
	Scrapping module for AC1102 : ACCOUNTING II 

Scanning couse: Accountancy Year 2 (GA)
	Scrapping module for AB0602 : COMMUNICATION MANAGEMENT STRATEGIES 
	Scrapping module for AB2000 : CAREER FOUNDATIONS II 
	Scrapping module for AC2101 : ACCOUNTING RECOGNITION & MEASUREMENT 
	Scrapping module for AC2104 : ASSURANCE & AUDITING 
	Scrapping module for AC2105 : ACCOUNTING FOR DECISION MAKING & CONTROL 
	Scrapping module for AC2301 : PRINCIPLES OF TAXATION* 
	Scrapping module for AC2302 : COMPANY LAW & CORPORATE GOVERNANCE* 
	Scrapping module for AC2401 : ACCOUNTING INFORMATION SYSTEMS* 
	Scrapping module for GC0001 : SUSTAINABILITY: SEEING THROUGH THE HAZE 
	Scrapping module for HY0001 : ETHICS & MORAL REASONING 

Scanning couse: Accountancy Year 3 (GA)
	Scrapping module for AB3601 : STRATEGIC MANAGEMENT* 
	Scrapping module for AC3102 : RISK REPORTING & ANALYSIS 
	Scrapping mod

	Scrapping module for DT3000 : ADVANCED DRAWN ANIMATION 
	Scrapping module for DV2009 : DESIGN IN MOTION* 
	Scrapping module for DV3003 : SPATIAL DESIGN* 
	Scrapping module for ET0001 : ENTERPRISE & INNOVATION 


WebDriverException: Message: chrome not reachable
  (Session info: chrome=74.0.3729.157)
  (Driver info: chromedriver=2.42.591088 (7b2b2dca23cca0862f674758c9a3933e685c27d5),platform=Windows NT 10.0.17134 x86_64)


### Checking timetable

In [207]:
collated_timetable

,INDEX,TYPE,GROUP,DAY,TIME,VENUE,REMARK,MODULE,NAME
1,LEC,LEC/STUDIO,1,TUE,1830-2130,LKC-LT,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
2,LEC,LEC/STUDIO,1,TUE,1830-2130,LT27,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
3,LEC,LEC/STUDIO,1,TUE,1830-2130,LT26,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
4,00822,SEM,1,WED,0830-1030,S4-CL1,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
8,00823,SEM,2,WED,1030-1230,S4-CL1,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
12,00824,SEM,3,THU,0830-1030,S4-CL1,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
16,00825,SEM,4,THU,0830-1030,S4-CL2,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
20,00826,SEM,5,THU,0830-1030,S4-CL3,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
24,00827,SEM,6,THU,0830-1030,S4-CL4,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
28,00828,SEM,7,THU,0830-1030,S4-CL5,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS


In [208]:
collated_timetable.to_csv('timetable.csv')

### Cleaning timetable

In [251]:
collated_timetable = pd.read_csv('timetable.csv')
collated_timetable.drop(['Unnamed: 0'], inplace = True, axis = 1)
collated_timetable

,INDEX,TYPE,GROUP,DAY,TIME,VENUE,REMARK,MODULE,NAME
0,LEC,LEC/STUDIO,1,TUE,1830-2130,LKC-LT,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
1,LEC,LEC/STUDIO,1,TUE,1830-2130,LT27,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
2,LEC,LEC/STUDIO,1,TUE,1830-2130,LT26,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
3,00822,SEM,1,WED,0830-1030,S4-CL1,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
4,00823,SEM,2,WED,1030-1230,S4-CL1,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
5,00824,SEM,3,THU,0830-1030,S4-CL1,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
6,00825,SEM,4,THU,0830-1030,S4-CL2,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
7,00826,SEM,5,THU,0830-1030,S4-CL3,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
8,00827,SEM,6,THU,0830-1030,S4-CL4,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
9,00828,SEM,7,THU,0830-1030,S4-CL5,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS


In [252]:
len(collated_timetable)

43776

In [253]:
collated_timetable.drop_duplicates(inplace = True)
len(collated_timetable)

6031

In [254]:
collated_timetable

,INDEX,TYPE,GROUP,DAY,TIME,VENUE,REMARK,MODULE,NAME
0,LEC,LEC/STUDIO,1,TUE,1830-2130,LKC-LT,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
1,LEC,LEC/STUDIO,1,TUE,1830-2130,LT27,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
2,LEC,LEC/STUDIO,1,TUE,1830-2130,LT26,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
3,00822,SEM,1,WED,0830-1030,S4-CL1,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
4,00823,SEM,2,WED,1030-1230,S4-CL1,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
5,00824,SEM,3,THU,0830-1030,S4-CL1,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
6,00825,SEM,4,THU,0830-1030,S4-CL2,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
7,00826,SEM,5,THU,0830-1030,S4-CL3,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
8,00827,SEM,6,THU,0830-1030,S4-CL4,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
9,00828,SEM,7,THU,0830-1030,S4-CL5,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS


In [255]:
collated_timetable.to_csv('timetable.csv')

### Checking info df

In [210]:
infos

[['AB0601 ', 'COMMUNICATION MANAGEMENT FUNDAMENTALS ', '2.0 AU'],
 ['AB0901 ', 'PRINCIPLES OF ECONOMICS: A SINGAPORE PERSPECTIVE* ', '3.0 AU'],
 ['AB1000 ', 'CAREER FOUNDATIONS I ', '1.0 AU'],
 ['AB1201 ', 'FINANCIAL MANAGEMENT* ', '3.0 AU'],
 ['AB1202 ', 'STATISTICS & ANALYSIS* ', '3.0 AU'],
 ['AB1301 ', 'BUSINESS LAW* ', '3.0 AU'],
 ['AB1401 ', 'TECHNOLOGICAL INNOVATIONS & DEVELOPMENTS* ', '3.0 AU'],
 ['AB1402 ', 'FOUNDATIONAL EXCEL* ', '1.0 AU'],
 ['AB1501 ', 'MARKETING* ', '3.0 AU'],
 ['AB1601 ', 'ORGANISATIONAL BEHAVIOUR & DESIGN* ', '3.0 AU'],
 ['AC1101 ', 'ACCOUNTING I ', '4.0 AU'],
 ['AC1102 ', 'ACCOUNTING II ', '4.0 AU'],
 ['AB0602 ', 'COMMUNICATION MANAGEMENT STRATEGIES ', '4.0 AU'],
 ['AB2000 ', 'CAREER FOUNDATIONS II ', '1.0 AU'],
 ['AC2101 ', 'ACCOUNTING RECOGNITION & MEASUREMENT ', '4.0 AU'],
 ['AC2104 ', 'ASSURANCE & AUDITING ', '4.0 AU'],
 ['AC2105 ', 'ACCOUNTING FOR DECISION MAKING & CONTROL ', '4.0 AU'],
 ['AC2301 ', 'PRINCIPLES OF TAXATION* ', '4.0 AU'],
 ['AC2302 ',

In [246]:
infos_df = pd.DataFrame(infos, columns = ['CODE', 'TITLE', 'AU', 'PREREQUISITE'])
infos_df

,CODE,TITLE,AU,PREREQUISITE
0,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,2.0 AU,None
1,AB0901,PRINCIPLES OF ECONOMICS: A SINGAPORE PERSPECTI...,3.0 AU,None
2,AB1000,CAREER FOUNDATIONS I,1.0 AU,None
3,AB1201,FINANCIAL MANAGEMENT*,3.0 AU,None
4,AB1202,STATISTICS & ANALYSIS*,3.0 AU,None
5,AB1301,BUSINESS LAW*,3.0 AU,None
6,AB1401,TECHNOLOGICAL INNOVATIONS & DEVELOPMENTS*,3.0 AU,AB1402(Corequisite)
7,AB1402,FOUNDATIONAL EXCEL*,1.0 AU,None
8,AB1501,MARKETING*,3.0 AU,None
9,AB1601,ORGANISATIONAL BEHAVIOUR & DESIGN*,3.0 AU,None


In [206]:
# C:\Users\JIanHui\Documents\_JIanHui\Projects\Python_Scripts\modules_info.csv
infos_df.to_csv('modules_info.csv')

### Cleaning modules_info df

In [247]:
infos_df = pd.read_csv('modules_info.csv')
infos_df.drop(['Unnamed: 0'], inplace = True, axis = 1)
print(len(infos_df))
infos_df

8664


,CODE,TITLE,AU,PREREQUISITE
0,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,2.0 AU,NaN
1,AB0901,PRINCIPLES OF ECONOMICS: A SINGAPORE PERSPECTIVE*,3.0 AU,NaN
2,AB1000,CAREER FOUNDATIONS I,1.0 AU,NaN
3,AB1201,FINANCIAL MANAGEMENT*,3.0 AU,NaN
4,AB1202,STATISTICS & ANALYSIS*,3.0 AU,NaN
5,AB1301,BUSINESS LAW*,3.0 AU,NaN
6,AB1401,TECHNOLOGICAL INNOVATIONS & DEVELOPMENTS*,3.0 AU,AB1402(Corequisite)
7,AB1402,FOUNDATIONAL EXCEL*,1.0 AU,NaN
8,AB1501,MARKETING*,3.0 AU,NaN
9,AB1601,ORGANISATIONAL BEHAVIOUR & DESIGN*,3.0 AU,NaN


In [248]:
infos_df.drop_duplicates(inplace = True)
print(len(infos_df))
infos_df

1161


,CODE,TITLE,AU,PREREQUISITE
0,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,2.0 AU,NaN
1,AB0901,PRINCIPLES OF ECONOMICS: A SINGAPORE PERSPECTIVE*,3.0 AU,NaN
2,AB1000,CAREER FOUNDATIONS I,1.0 AU,NaN
3,AB1201,FINANCIAL MANAGEMENT*,3.0 AU,NaN
4,AB1202,STATISTICS & ANALYSIS*,3.0 AU,NaN
5,AB1301,BUSINESS LAW*,3.0 AU,NaN
6,AB1401,TECHNOLOGICAL INNOVATIONS & DEVELOPMENTS*,3.0 AU,AB1402(Corequisite)
7,AB1402,FOUNDATIONAL EXCEL*,1.0 AU,NaN
8,AB1501,MARKETING*,3.0 AU,NaN
9,AB1601,ORGANISATIONAL BEHAVIOUR & DESIGN*,3.0 AU,NaN


In [34]:
infos_df.isna().sum()

CODE              0
TITLE             0
AU                0
PREREQUISITE    597
dtype: int64

In [249]:
infos_df.to_csv('modules_info.csv')

# Simulating for Accountancy (GA) Year 2

### Intialise

In [234]:
driver = webdriver.Chrome()
driver.get('https://wish.wis.ntu.edu.sg/webexe/owa/aus_schedule.main')
optElem = driver.find_elements_by_tag_name('option')
courses = optElem[21:-1]

### Core code

In [235]:
# click course
driver.find_element_by_xpath('//*[@id="ui_body_container"]/table/tbody/tr/td[2]/select[2]/option[3]').click()
# click load
driver.find_element_by_xpath('//*[@id="ui_body_container"]/table/tbody/tr/td[2]/input').click()
# switch tab to table
driver.switch_to.window(driver.window_handles[1])

# time.sleep(5)
# driver.close()

# wtf is this
## driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + 't')

### Grabbing info table

In [236]:
# info table at top of each module
df = pd.read_html(driver.find_elements_by_tag_name('table')[0].get_attribute('outerHTML'))[0]
df

,0,1,2
0,AB0602,COMMUNICATION MANAGEMENT STRATEGIES,4.0 AU
1,Prerequisite:,AB114 OR,NaN
2,NaN,AB0601,NaN


In [237]:
list(df.iloc[0])

['AB0602', 'COMMUNICATION MANAGEMENT STRATEGIES', '4.0 AU']

In [232]:
list(df.iloc[0])[1]

'COMMUNICATION MANAGEMENT FUNDAMENTALS'

In [242]:
df[1].iloc[1:]

1    AB114 OR
2      AB0601
Name: 1, dtype: object

In [241]:
header = ['CODE', 'TITLE', 'AU', 'PREREQUISITE']
infos = []

# retiterate this code
info = list(df.iloc[0])
info.append(' '.join(df[1].iloc[1:]))
# print(info)
infos.append(info)
# print(infos)
infos_df = pd.DataFrame(infos, columns = header)
infos_df

,CODE,TITLE,AU,PREREQUISITE
0,AB0602,COMMUNICATION MANAGEMENT STRATEGIES,4.0 AU,AB114 OR AB0601


In [126]:
' '.join(df[1].iloc[1:])

'AB114 OR AB0601'

In [65]:
test = (['AB0602', 'COMMUNICATION MANAGEMENT STRATEGIES', '4.0 AU', 'AB114 OR AB0601'],'zero')
test

(['AB0602',
  'COMMUNICATION MANAGEMENT STRATEGIES',
  '4.0 AU',
  'AB114 OR AB0601'],
 'zero')

In [66]:
pd.MultiIndex.from_tuples(test)

MultiIndex(levels=[['AB0602', 'z'], ['COMMUNICATION MANAGEMENT STRATEGIES', 'e'], ['4.0 AU', 'r'], ['AB114 OR AB0601', 'o']],
           labels=[[0, 1], [0, 1], [0, 1], [0, 1]])

### Grabbing timetable column

In [103]:
# read table
timetable = pd.read_html(driver.find_elements_by_tag_name('table')[1].get_attribute('outerHTML'))[0]

# replace 1st column as headers
timetable.columns = timetable.iloc[0]
timetable.drop([0], axis = 0, inplace = True)

# fill all na in INDEX and REMARK
timetable.fillna(method='ffill', inplace = True)

# drop all lectures since duplicates
timetable.drop_duplicates(subset = ['TYPE','GROUP','DAY','TIME','VENUE'], inplace = True)


timetable

,INDEX,TYPE,GROUP,DAY,TIME,VENUE,REMARK
1,00861,LEC/STUDIO,1,THU,1830-2130,LT26,Teaching Wk12
2,00861,LEC/STUDIO,1,THU,1830-2130,LKC-LT,Teaching Wk12
3,00861,SEM,1,THU,1230-1430,S4-CL1,Teaching Wk12
4,00861,SEM,1,MON,1230-1430,S4-CL1,Teaching Wk12
7,00862,SEM,2,MON,1230-1430,S4-CL2,Teaching Wk12
8,00862,SEM,2,THU,1230-1430,S4-CL2,Teaching Wk12
11,00863,SEM,3,THU,1230-1430,S4-CL3,Teaching Wk12
12,00863,SEM,3,MON,1230-1430,S4-CL3,Teaching Wk12
15,00864,SEM,4,MON,1230-1430,S4-CL4,Teaching Wk12
16,00864,SEM,4,THU,1230-1430,S4-CL4,Teaching Wk12


In [138]:
timetable['MODULE'] = 'AB0602'
timetable.loc[timetable.TYPE == 'LEC/STUDIO', 'INDEX'] = 'LEC'

In [139]:
timetable

,INDEX,TYPE,GROUP,DAY,TIME,VENUE,REMARK
1,LEC,LEC/STUDIO,1,THU,1830-2130,LT26,Teaching Wk12
2,LEC,LEC/STUDIO,1,THU,1830-2130,LKC-LT,Teaching Wk12
3,00861,SEM,1,THU,1230-1430,S4-CL1,Teaching Wk12
4,00861,SEM,1,MON,1230-1430,S4-CL1,Teaching Wk12
7,00862,SEM,2,MON,1230-1430,S4-CL2,Teaching Wk12
8,00862,SEM,2,THU,1230-1430,S4-CL2,Teaching Wk12
11,00863,SEM,3,THU,1230-1430,S4-CL3,Teaching Wk12
12,00863,SEM,3,MON,1230-1430,S4-CL3,Teaching Wk12
15,00864,SEM,4,MON,1230-1430,S4-CL4,Teaching Wk12
16,00864,SEM,4,THU,1230-1430,S4-CL4,Teaching Wk12


In [98]:
timetable[timetable.INDEX == '00862']

,INDEX,TYPE,GROUP,DAY,TIME,VENUE,REMARK
7,00862,SEM,2,THU,1230-1430,S4-CL2,Teaching Wk12
8,00862,SEM,2,MON,1230-1430,S4-CL2,Teaching Wk12


In [1]:
testdf = timetable.groupby('TYPE').agg(lambda x: tuple(x))
testdf

NameError: name 'timetable' is not defined

In [184]:
testdf['grouped'] = testdf.GROUP + testdf.DAY + testdf.TIME + testdf.VENUE + testdf.REMARK
testdf

,GROUP,DAY,TIME,VENUE,REMARK,grouped
TYPE,,,,,,
LEC/STUDIO,"(1, 1)","(TUE, TUE)","(1830-2130, 1830-2130)","(LT26, LKC-LT)","(Teaching Wk13, Teaching Wk13)","(1, 1, TUE, TUE, 1830-2130, 1830-2130, LT26, L..."
SEM,"(1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, ...","(THU, MON, MON, THU, THU, MON, MON, THU, THU, ...","(1230-1430, 1230-1430, 1230-1430, 1230-1430, 1...","(S4-CL1, S4-CL1, S4-CL2, S4-CL2, S4-CL3, S4-CL...","(Teaching Wk13, Teaching Wk13, Teaching Wk13, ...","(1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, ..."
